In [ ]:
pip install PyMuPDF pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 80.8 MB/s eta 0:00:00


In [37]:
import fitz
import pandas as pd
import os

def extract_tables_from_pdf(pdf_path):
    """
    Extracts tables from a PDF file.
    Args:
        pdf_path (str): Path to the PDF file.
    Returns:
        list: List of tables, where each table is a list of rows.
    """
    # Opens the PDF file
    pdf_document = fitz.open(pdf_path)
    tables = []

    # Iterating through each page
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        words = page.get_text("words")  # Extract words with their coordinates

        # Grouping the words into lines based on their y-coordinates
        lines = {}
        for word in words:
            x0, y0, x1, y1, word_text, block_no, line_no, word_no = word
            if y0 not in lines:
                lines[y0] = []
            lines[y0].append((x0, word_text))

        # Sorting lines by y-coordinate
        sorted_lines = sorted(lines.items(), key=lambda x: x[0])

        # Converting lines into a table structure
        table = []
        for _, line_words in sorted_lines:
            # Sorting words in the line by x-coordinate
            sorted_words = sorted(line_words, key=lambda x: x[0])
            row = [word[1] for word in sorted_words]
            table.append(row)

        # Adds the table to the list of tables
        if table:
            tables.append(table)

    return tables

def save_tables_to_csv(tables, output_csv_folder):
    """
    Saves extracted tables to CSV files.
    Args: tables (list): List of tables, where each table is a list of rows. output_csv_folder (str): Path to the folder where CSV files will be saved.
    """
    # Createing the output folder if it doesn't exist already
    if not os.path.exists(output_csv_folder):
        os.makedirs(output_csv_folder)

    # Saveing each table as a separate CSV file
    for i, table in enumerate(tables):
        # Converting the table to a pandas DataFrame
        df = pd.DataFrame(table)
        # Saveing the DataFrame to a CSV file
        csv_path = os.path.join(output_csv_folder, f"Table_{i+1}.csv")
        df.to_csv(csv_path, index=False, header=False)
        print(f"Table {i+1} saved to {csv_path}")

def main(pdf_path, output_csv_folder):
    """
    Main function to extract tables from a PDF and save them to CSV files.
    Args:
        pdf_path (str): Path to the input PDF file.
        output_csv_folder (str): Path to save the output CSV files.
    """
    # Extract tables from the PDF
    tables = extract_tables_from_pdf(pdf_path)
    # Save the tables to CSV files
    save_tables_to_csv(tables, output_csv_folder)
    print(f"Tables extracted and saved to {output_csv_folder}")

# Example usage
pdf_path = "test3.pdf"  # PDF file path
output_csv_folder = "output_csv"  # CSV files
main(pdf_path, output_csv_folder)

Table 1 saved to output_csv/Table_1.csv
Table 2 saved to output_csv/Table_2.csv
Table 3 saved to output_csv/Table_3.csv
Table 4 saved to output_csv/Table_4.csv
Table 5 saved to output_csv/Table_5.csv
Table 6 saved to output_csv/Table_6.csv
Table 7 saved to output_csv/Table_7.csv
Table 8 saved to output_csv/Table_8.csv
Tables extracted and saved to output_csv


Validation Part


In [40]:
import pandas as pd
import os

def calculate_accuracy(extracted_tables, reference_folder):
    """
    Compares extracted tables with manually verified reference tables to calculate accuracy.

    Args:
        extracted_tables (list): List of extracted tables (each table is a list of rows).
        reference_folder (str): Path to the folder containing manually verified reference CSV files.

    Returns:
        dict: Accuracy metrics (cell-level, row-level, table-level).
    """
    total_cells = 0
    correct_cells = 0
    total_rows = 0
    correct_rows = 0
    total_tables = len(extracted_tables)
    correct_tables = 0

    for i, extracted_table in enumerate(extracted_tables):
        csv_path = os.path.join(reference_folder, f"Table_{i+1}.csv")

        if not os.path.exists(csv_path):
            print(f"Reference file {csv_path} not found. Skipping comparison for this table.")
            continue

        # Load reference table
        reference_df = pd.read_csv(csv_path, header=None)
        extracted_df = pd.DataFrame(extracted_table)

        # Convert both DataFrames to string format for direct comparison
        reference_values = reference_df.astype(str).values
        extracted_values = extracted_df.astype(str).values

        # Get table dimensions
        ref_rows, ref_cols = reference_values.shape
        ext_rows, ext_cols = extracted_values.shape

        # Ensure shape consistency by padding with empty strings
        max_rows = max(ref_rows, ext_rows)
        max_cols = max(ref_cols, ext_cols)
        padded_reference = [[""] * max_cols for _ in range(max_rows)]
        padded_extracted = [[""] * max_cols for _ in range(max_rows)]

        for r in range(ref_rows):
            for c in range(ref_cols):
                padded_reference[r][c] = reference_values[r][c]

        for r in range(ext_rows):
            for c in range(ext_cols):
                padded_extracted[r][c] = extracted_values[r][c]

        # cell-level
        for r in range(max_rows):
            for c in range(max_cols):
                total_cells += 1
                if padded_reference[r][c] == padded_extracted[r][c]:
                    correct_cells += 1

        # row-level
        for r in range(max_rows):
            total_rows += 1
            if padded_reference[r] == padded_extracted[r]:
                correct_rows += 1

        #table-level
        if reference_df.shape == extracted_df.shape and reference_df.equals(extracted_df):
            correct_tables += 1

    cell_accuracy = correct_cells / total_cells * 100 if total_cells else 0
    row_accuracy = correct_rows / total_rows * 100 if total_rows else 0
    table_accuracy = correct_tables / total_tables * 100 if total_tables else 0

    accuracy_metrics = {
        "Cell-Level Accuracy": f"{cell_accuracy:.2f}%",
        "Row-Level Accuracy": f"{row_accuracy:.2f}%",
        "Table-Level Accuracy": f"{table_accuracy:.2f}%"
    }

    return accuracy_metrics